<a href="https://colab.research.google.com/github/xiashenzhen/geemap/blob/main/%E7%B2%BE%E5%BA%A6%E8%AF%84%E4%BB%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

精度评价

In [5]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-jason2024')

In [ ]:
Map = geemap.Map()
point = ee.Geometry.Point([-122.4439, 37.7538])

img = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2021-01-01')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
    .select('B.*')
)

vis_params = {'min': 100, 'max': 3500, 'bands': ['B11',  'B8',  'B3']}

Map.centerObject(point, 9)
Map.addLayer(img, vis_params, "Sentinel-2")
Map

In [7]:
lc = ee.Image('ESA/WorldCover/v100/2020')
classValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
remapValues = ee.List.sequence(0, 10)
label = 'lc'
lc = lc.remap(classValues, remapValues).rename(label).toByte()

In [10]:
sample = img.addBands(lc).stratifiedSample(**{
  'numPoints': 100,
  'classBand': label,
  'region': img.geometry(),
  'scale': 10,
  'geometries': True
})

In [11]:
sample = sample.randomColumn()
trainingSample = sample.filter('random <= 0.8')
validationSample = sample.filter('random > 0.8')

In [12]:
trainedClassifier = ee.Classifier.smileRandomForest(numberOfTrees=10).train(**{
  'features': trainingSample,
  'classProperty': label,
  'inputProperties': img.bandNames()
})

In [ ]:
trainedClassifier.explain().getInfo()

In [15]:
trainAccuracy = trainedClassifier.confusionMatrix()
trainAccuracy.getInfo()

[[76, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 80, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 85, 0, 0, 0, 0, 0, 0],
 [0, 1, 4, 75, 0, 3, 0, 0, 0],
 [0, 0, 1, 0, 79, 0, 0, 0, 1],
 [0, 1, 2, 1, 3, 75, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 76, 0],
 [2, 1, 0, 0, 0, 0, 0, 0, 72]]

In [16]:
trainAccuracy.accuracy().getInfo()

0.965625

In [17]:
trainAccuracy.kappa().getInfo()

0.9607007985441431

In [18]:
validationSample = validationSample.classify(trainedClassifier)
validationAccuracy = validationSample.errorMatrix(label, 'classification')
validationAccuracy.getInfo()

[[16, 1, 3, 1, 3, 0, 0, 0, 0],
 [1, 15, 1, 0, 0, 2, 0, 0, 0],
 [2, 2, 10, 0, 0, 0, 0, 0, 0],
 [0, 0, 4, 9, 1, 3, 0, 0, 0],
 [3, 0, 0, 0, 12, 4, 0, 0, 0],
 [1, 4, 1, 2, 2, 4, 0, 0, 4],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 24, 0],
 [3, 2, 0, 1, 0, 1, 0, 0, 18]]

In [19]:
validationAccuracy.accuracy().getInfo()

0.675

In [20]:
validationAccuracy.producersAccuracy().getInfo()

[[0.6666666666666666],
 [0.7894736842105263],
 [0.7142857142857143],
 [0.5294117647058824],
 [0.631578947368421],
 [0.2222222222222222],
 [0],
 [1],
 [0.72]]

In [21]:
validationAccuracy.consumersAccuracy().getInfo()

[[0.6153846153846154,
  0.625,
  0.5263157894736842,
  0.6923076923076923,
  0.6666666666666666,
  0.2857142857142857,
  0,
  1,
  0.8181818181818182]]

In [22]:
import csv

with open("training.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(trainAccuracy.getInfo())

with open("validation.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(validationAccuracy.getInfo())

In [23]:
imgClassified = img.classify(trainedClassifier)

In [ ]:
classVis = {
  'min': 0,
  'max': 10,
  'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4',
            'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0']
}
Map.addLayer(lc, classVis, 'ESA Land Cover', False)
Map.addLayer(imgClassified, classVis, 'Classified')
Map.addLayer(trainingSample, {'color': 'black'}, 'Training sample')
Map.addLayer(validationSample, {'color': 'white'}, 'Validation sample')
Map.add_legend(title='Land Cover Type', builtin_legend='ESA_WorldCover')
Map.centerObject(img)
Map